# Gill-Matsuno model

The Gill-Matsuno model is a classical atmospheric model that describes the tropical atmospheric response to a prescribed heating. It consists of 3 prognostic variables $u$, $v$, and $T$ in an equatorial beta-plane approximation. The non-dimensional form of the equations is given by:

$$
\frac{\partial u}{\partial t} + eu - \frac{1}{2} yv = -\frac{\partial \phi}{\partial x}
$$
$$
\frac{\partial v}{\partial t} + ev + \frac{1}{2} xu = -\frac{\partial \phi}{\partial y}
$$
$$
\frac{\partial \phi}{\partial t} + e\phi + \frac{\partial u}{\partial x} + \frac{\partial v}{\partial y} = Q
$$

For the Gill-Matsuno model, we look for the steady-state solution, which means we set the time derivatives to zero, further simplifying the equations. This set of equations represents the balance between the horizontal momentum equations and the continuity equation in the context of a tropical atmosphere.

$$
eu - \frac{1}{2} yv = -\frac{\partial \phi}{\partial x}
$$
$$
ev + \frac{1}{2} xu = -\frac{\partial \phi}{\partial y}
$$
$$
e\phi + \frac{\partial u}{\partial x} + \frac{\partial v}{\partial y} = Q
$$
